In [1]:
import torch,torchvision
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
import numpy as np
import pandas as pd
import wandb
import os, json, cv2, random
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor,DefaultTrainer
from detectron2.config import get_cfg
from detectron2.structures import BoxMode
from tqdm import tqdm
import matplotlib.pyplot as plt
from detectron2.utils.visualizer import ColorMode
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
PROJECT_NAME = 'Car-Object-Detection-V2-Learning-Detectron2-V2'

In [2]:
data = pd.read_csv('./data.csv')

In [4]:
xmin,ymin,xmax,ymax = 281,187,327,223

In [5]:
x = xmin
y = ymin
w = xmax - xmin
h = ymax - ymin

In [6]:
data

,image,xmin,ymin,xmax,ymax
0,vid_4_1000.jpg,281.259045,187.035071,327.727931,223.225547
1,vid_4_10000.jpg,15.163531,187.035071,120.329957,236.430180
2,vid_4_10040.jpg,239.192475,176.764801,361.968162,236.430180
3,vid_4_10020.jpg,496.483358,172.363256,630.020260,231.539575
4,vid_4_10060.jpg,16.630970,186.546010,132.558611,238.386422
...,...,...,...,...,...
554,vid_4_9860.jpg,0.000000,198.321729,49.235251,236.223284
555,vid_4_9880.jpg,329.876184,156.482351,536.664239,250.497895
556,vid_4_9900.jpg,0.000000,168.295823,141.797524,239.176652
557,vid_4_9960.jpg,487.428988,172.233646,616.917699,228.839864


In [7]:
img = cv2.imread('./data/vid_4_1000.jpg')

In [8]:
img_crop = img[y:y+h,x:x+w]

In [10]:
cv2.imwrite('./crop.png',img_crop)

True

In [12]:
img = cv2.rectangle(img,(x,y),(x+w,y+h),(200,0,0),2)

In [13]:
cv2.imwrite('./box.png',img)

True

In [ ]:
def load_data():
    new_data = []
    idx = len(data)
    for i in range(idx):
        record = {}
        info = data.iloc[i]
        img_path = f"./data/{info['image']}"
        height,width = cv2.imread(img_path).shape[:2]
        record['height'] = height
        record['width'] = width
        record['category_id'] = 0
        record['file_name'] = img_path
        record['image_id'] = i
        record['annotaions'] = [{'bbox':[info['xmin'],info['ymin'],info['xmax'],info['ymaxx']],'bbox_mode':BoxMode.XYXY_ABS,'category_id':0}]
        new_data.append(record)
    return new_data

In [14]:
labels = ['car']

In [ ]:
DatasetCatalog.register('data', lambda : load_data())
MetadataCatalog.get('data').set(thing_classes=len(labels))
metadata = MetadataCatalog.get('data')

In [ ]:
wandb.init(project=PROJECT_NAME,name='baseline')
model = 'COCO-Detection/faster_rcnn_R_101_C4_3x.yaml'
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config(model))
cfg.DATASETS.TRAIN = ('data',)
cfg.DATASET.TEST = ()
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 250
cfg.SOLVER.IMS_PER_BATCH = 128
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url('COCO-Detection/faster_rcnn_R_101_C4_3x.yaml')
cfg.SOLVER.STEPS = []
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()
evaluator = COCOEvaluator("data", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "data")
wandb.log(inference_on_dataset(predictor.model, val_loader, evaluator))
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
predictor = DefaultPredictor(cfg)

In [ ]:
im = cv2.imread('./data/vid_4_1000.jpg')
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1],
                metadata=metadata, 
               scale=0.5, 
               instance_mode=ColorMode.IMAGE_BW
              )
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
plt.figure(figsize=(12,6))
plt.imshow(out.get_image()[:, :, ::-1])
plt.savefig('./pred.png')
plt.close()